In [1]:
import sys
sys.path.append("../")

import os
import pandas as pd
from utils import get_digit_and_unit
from collections import OrderedDict

base_path = "/workspace/Data"
data_fn = "20240327_numeric_dataset_from_claude.xlsx"

In [2]:
raw_data = pd.read_excel(os.path.join(base_path, data_fn), engine='openpyxl', header=None)
raw_data.columns = ["data"]
raw_data

,data
0,"[5.3kg = 5300g, 9.8m = 980cm, 0.058km = 58m, 2..."
1,"[4.5kg = 4500g, 8.2m = 820cm, 0.055km = 55m, 1..."
2,"[5.7kg = 5700g, 9.5m = 950cm, 0.061km = 61m, 2..."
3,"[4.8kg = 4800g, 7.6m = 760cm, 0.056km = 56m, 1..."
4,"[6.7kg = 6700g, 8.7m = 870cm, 0.038km = 38m, 2..."
...,...
506,"[1.2345kg = 1234.5g, 0.0123m = 1.23cm, 0.0456k..."
507,"[12.3456kg = 12345.6g, 0.00123m = 0.123cm, 0.0..."
508,"[321.098kg = 321098g, 65.4321m = 6543.21cm, 0...."
509,"[6543210g, 12.3456m = 1234.56cm, 0.00123km = 1..."


In [7]:
data = []
for idx, row in raw_data.iterrows():
    temp_row = row[0][1:-1].split(", ")
    data.extend(temp_row)

data_df = []
for item in data:
    
    if "=" in item:
        parts = item.split("=")
        data_df.append([parts[0].strip(), "=", parts[1].strip()])
        data_df.append([parts[1].strip(), "=", parts[0].strip()])
    elif ">" in item:
        parts = item.split(">")
        data_df.append([parts[0].strip(), ">", parts[1].strip()])
        data_df.append([parts[1].strip(), "<", parts[0].strip()])
    elif "<" in item:
        parts = item.split("<")
        data_df.append([parts[0].strip(), "<", parts[1].strip()])
        data_df.append([parts[1].strip(), ">", parts[0].strip()])

data_df = pd.DataFrame(data_df, columns=["value1", "operation", "value2"])
data_df

,value1,operation,value2
0,5.3kg,=,5300g
1,5300g,=,5.3kg
2,9.8m,=,980cm
3,980cm,=,9.8m
4,0.058km,=,58m
...,...,...,...
10325,0.0003210987l,=,0.3210987ml
10326,0.2109876ml,=,0.0002109876l
10327,0.0002109876l,=,0.2109876ml
10328,0.1098765ml,=,0.0001098765l


In [9]:
# Load Data From Generation

data_v2_path = "../data/numeric_embedding_data_v2.xlsx"
data_v2 = pd.read_excel(data_v2_path, engine='openpyxl')
data_v2

,value1,operation,value2
0,5.099g,=,5099.0mg
1,77.429934l,=,77429.934ml
2,0.012443km,=,12443.0mm
3,29396.1cm,=,293961.0mm
4,3.282787kg,=,3282787.0mg
...,...,...,...
133552,4.185007l,=,4185.007ml
133553,77.343m,=,7734.3cm
133554,8.891m,=,8891.0mm
133555,25.699m,=,2569.9cm


In [10]:
data_df = pd.concat([data_df, data_v2])
data_df.reset_index(drop=True, inplace=True)
data_df

,value1,operation,value2
0,5.3kg,=,5300g
1,5300g,=,5.3kg
2,9.8m,=,980cm
3,980cm,=,9.8m
4,0.058km,=,58m
...,...,...,...
143882,4.185007l,=,4185.007ml
143883,77.343m,=,7734.3cm
143884,8.891m,=,8891.0mm
143885,25.699m,=,2569.9cm


## Preprocess

In [11]:
# drop duplicated

data_df.drop_duplicates(inplace=True)
data_df.reset_index(drop=True, inplace=True)
data_df

,value1,operation,value2
0,5.3kg,=,5300g
1,5300g,=,5.3kg
2,9.8m,=,980cm
3,980cm,=,9.8m
4,0.058km,=,58m
...,...,...,...
137564,4.185007l,=,4185.007ml
137565,77.343m,=,7734.3cm
137566,8.891m,=,8891.0mm
137567,25.699m,=,2569.9cm


In [12]:
# invalid case1

invalid_idx = []
max_digit_length = 5
for idx, row in data_df.iterrows():
    value1, unit1 = get_digit_and_unit(row['value1'])
    value2, unit2 = get_digit_and_unit(row['value2'])
    operation = row['operation']
    
    if operation == "=" and (unit1 == unit2):
        invalid_idx.append(idx)

data_df.loc[invalid_idx]

,value1,operation,value2
726,0.052m,=,52m
727,52m,=,0.052m
728,0.037m,=,37m
729,37m,=,0.037m
730,0.077m,=,77m
731,77m,=,0.077m
732,0.042m,=,42m
733,42m,=,0.042m
734,0.033m,=,33m
735,33m,=,0.033m


In [13]:
data_df.drop(invalid_idx, inplace=True)
data_df.reset_index(drop=True, inplace=True)
data_df

,value1,operation,value2
0,5.3kg,=,5300g
1,5300g,=,5.3kg
2,9.8m,=,980cm
3,980cm,=,9.8m
4,0.058km,=,58m
...,...,...,...
137542,4.185007l,=,4185.007ml
137543,77.343m,=,7734.3cm
137544,8.891m,=,8891.0mm
137545,25.699m,=,2569.9cm


In [15]:
# invalid case2
# The desired pair is following: (kg, g, mg), (km, m, cm, mm), (l, ml)

invalid_idx = []
desired_unit_group = {
    "kg": "weight",
    "g": "weight",
    "mg": "weight",
    "km": "length",
    "m": "length",
    "cm": "length",
    "mm": "length",
    "l": "volume",
    "ml": "volume",
}

for idx, row in data_df.iterrows():
    value1, unit1 = get_digit_and_unit(row['value1'])
    value2, unit2 = get_digit_and_unit(row['value2'])
    
    if desired_unit_group[unit1] != desired_unit_group[unit2]:
        invalid_idx.append(idx)

data_df.loc[invalid_idx]

,value1,operation,value2
590,0.053kg,=,53m
591,53m,=,0.053kg
596,0.058kg,=,58m
597,58m,=,0.058kg
598,0.032kg,=,32m
599,32m,=,0.032kg
600,0.07kg,=,70m
601,70m,=,0.07kg
602,0.035kg,=,35m
603,35m,=,0.035kg


In [16]:
data_df.drop(invalid_idx, inplace=True)
data_df.reset_index(drop=True, inplace=True)
data_df

,value1,operation,value2
0,5.3kg,=,5300g
1,5300g,=,5.3kg
2,9.8m,=,980cm
3,980cm,=,9.8m
4,0.058km,=,58m
...,...,...,...
137488,4.185007l,=,4185.007ml
137489,77.343m,=,7734.3cm
137490,8.891m,=,8891.0mm
137491,25.699m,=,2569.9cm


In [17]:
# invalid case3
# Categorize the digit
def categorize_digit(digit):
    if 0 <= digit < 1e-10:
        return "[0, 1e-10]"
    elif 1e-10 <= digit < 1e-9:
        return "[1e-10, 1e-9]"
    elif 1e-9 <= digit < 1e-8:
        return "[1e-9, 1e-8]"
    elif 1e-8 <= digit < 1e-7:
        return "[1e-8, 1e-7]"
    elif 1e-7 <= digit < 1e-6:
        return "[1e-7, 1e-6]"
    elif 1e-6 <= digit < 1e-5:
        return "[1e-6, 1e-5]"
    elif 1e-5 <= digit < 1e-4:
        return "[1e-5, 1e-4]"
    elif 1e-4 <= digit < 1e-3:
        return "[1e-4, 1e-3]"
    elif 1e-3 <= digit < 1e-2:
        return "[1e-3, 1e-2]"
    elif 1e-2 <= digit < 1e-1:
        return "[1e-2, 1e-1]"
    elif 1e-1 <= digit < 1:
        return "[1e-1, 1]"
    elif 1 <= digit < 10:
        return "[1, 10]"
    elif 10 <= digit < 100:
        return "[10, 100]"
    elif 100 <= digit < 1000:
        return "[100, 1000]"
    elif 1000 <= digit < 10000:
        return "[1000, 10000]"
    elif 10000 <= digit < 100000:
        return "[10000, 100000]"
    elif 100000 <= digit < 1000000:
        return "[100000, 1000000]"
    elif 1000000 <= digit < 1e7:
        return "[1000000, 1e7]"
    elif 1e7 <= digit < 1e8:
        return "[1e7, 1e8]"
    elif 1e8 <= digit < 1e9:
        return "[1e8, 1e9]"
    elif 1e9 <= digit < 1e10:
        return "[1e9, 1e10]"
    else:
        return "[1e10, ..]"

digit_range_count = OrderedDict([
    ("[0, 1e-10]", 0),
    ("[1e-10, 1e-9]", 0),
    ("[1e-9, 1e-8]", 0),
    ("[1e-8, 1e-7]", 0),
    ("[1e-7, 1e-6]", 0),
    ("[1e-6, 1e-5]", 0),
    ("[1e-5, 1e-4]", 0),
    ("[1e-4, 1e-3]", 0),
    ("[1e-3, 1e-2]", 0),
    ("[1e-2, 1e-1]", 0),
    ("[1e-1, 1]", 0),
    ("[1, 10]", 0),
    ("[10, 100]", 0),
    ("[100, 1000]", 0),
    ("[1000, 10000]", 0),
    ("[10000, 100000]", 0),
    ("[100000, 1000000]", 0),
    ("[1000000, 1e7]", 0),
    ("[1e7, 1e8]", 0),
    ("[1e8, 1e9]", 0),
    ("[1e9, 1e10]", 0),
    ("[1e10, ..]", 0)
])

for idx, row in data_df.iterrows():
    value1, unit1 = get_digit_and_unit(row['value1'])
    value2, unit2 = get_digit_and_unit(row['value2'])
    
    for digit in [value1, value2]:
        category = categorize_digit(digit)
        digit_range_count[category] += 1
        
# Print the count for each digit range
for range_str, count in digit_range_count.items():
    print(f"{range_str}: {count}")

[0, 1e-10]: 0
[1e-10, 1e-9]: 0
[1e-9, 1e-8]: 0
[1e-8, 1e-7]: 0
[1e-7, 1e-6]: 0
[1e-6, 1e-5]: 0
[1e-5, 1e-4]: 0
[1e-4, 1e-3]: 2406
[1e-3, 1e-2]: 15035
[1e-2, 1e-1]: 18201
[1e-1, 1]: 20527
[1, 10]: 32695
[10, 100]: 30352
[100, 1000]: 35165
[1000, 10000]: 37264
[10000, 100000]: 32282
[100000, 1000000]: 27907
[1000000, 1e7]: 12286
[1e7, 1e8]: 6362
[1e8, 1e9]: 4504
[1e9, 1e10]: 0
[1e10, ..]: 0


In [18]:
invalid_idx = []
max_digit_length = 6
for idx, row in data_df.iterrows():
    value1, unit1 = get_digit_and_unit(row['value1'])
    value2, unit2 = get_digit_and_unit(row['value2'])
    
    if len(str(value1).split(".")[0]) > max_digit_length or len(str(value2).split(".")[0]) > max_digit_length:
        invalid_idx.append(idx)
        
    if len(str(value1).split(".")[1]) > max_digit_length or len(str(value2).split(".")[1]) > max_digit_length:
        invalid_idx.append(idx)

data_df.loc[invalid_idx]

,value1,operation,value2
760,5.874km,=,5874000mm
761,5874000mm,=,5.874km
776,1.093km,=,1093000mm
777,1093000mm,=,1.093km
806,3.178km,=,3178000mm
...,...,...,...
137446,3.773196kg,=,3773196.0mg
137462,3.805018kg,=,3805018.0mg
137466,11651.175g,=,11651175.0mg
137475,9944.489g,=,9944489.0mg


In [19]:
data_df.drop(invalid_idx, inplace=True)
data_df.reset_index(drop=True, inplace=True)
data_df

,value1,operation,value2
0,5.3kg,=,5300g
1,5300g,=,5.3kg
2,9.8m,=,980cm
3,980cm,=,9.8m
4,0.058km,=,58m
...,...,...,...
114252,4.185007l,=,4185.007ml
114253,77.343m,=,7734.3cm
114254,8.891m,=,8891.0mm
114255,25.699m,=,2569.9cm


## EDA

In [20]:
# Check the ratio of (kg, g, mg) (km, m, cm, mm) (l, ml)

weight_data, length_data, volume_data, error_data = [], [], [], []

for idx, row in data_df.iterrows():
    value1, unit1 = get_digit_and_unit(row['value1'])
    value2, unit2 = get_digit_and_unit(row['value2'])
    
    if unit1 in ["kg", "g", "mg"] and unit2 in ["kg", "g", "mg"]:
        weight_data.append(idx)
    elif unit1 in ["km", "m", "cm", "mm"] and unit2 in ["km", "m", "cm", "mm"]:
        length_data.append(idx)
    elif unit1 in ["l", "ml"] and unit2 in ["l", "ml"]:
        volume_data.append(idx)
    else:
        error_data.append(idx)

print("Weight: ", len(weight_data))
print("Length: ", len(length_data))
print("Volume: ", len(volume_data))
print("Error: ", len(error_data))

Weight:  37573
Length:  58542
Volume:  18142
Error:  0


In [21]:
# Ratio of Equal/Greater/Less

equal_data = data_df[data_df["operation"] == "="].sort_values(by=["value1", "value2"]).reset_index(drop=True)
greater_data = data_df[data_df["operation"] == ">"].sort_values(by=["value1", "value2"]).reset_index(drop=True)
less_data = data_df[data_df["operation"] == "<"].sort_values(by=["value1", "value2"]).reset_index(drop=True)

print("Equal: ", equal_data.shape[0])
print("Greater: ", greater_data.shape[0])
print("Less: ", less_data.shape[0])

Equal:  112883
Greater:  687
Less:  687


## Save the data

In [22]:
equal_data = data_df[data_df["operation"] == "="].sort_values(by=["value1", "value2"]).reset_index(drop=True)
greater_data = data_df[data_df["operation"] == ">"].sort_values(by=["value1", "value2"]).reset_index(drop=True)
less_data = data_df[data_df["operation"] == "<"].sort_values(by=["value1", "value2"]).reset_index(drop=True)

In [23]:
# Save the data into excel file with different sheets name: equal, greater, less

output_path = "../data/umeric_embedding_data_v3.xlsx"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with pd.ExcelWriter(output_path) as writer:
    equal_data.to_excel(writer, sheet_name='equal', index=False, engine='openpyxl')
    greater_data.to_excel(writer, sheet_name='greater', index=False, engine='openpyxl')
    less_data.to_excel(writer, sheet_name='less', index=False, engine='openpyxl')